# Esercitazione Pratica: Tecniche di Riduzione della Dimensionalità

## Corso di Machine Learning: Apprendimento Non Supervisionato

Questa esercitazione pratica ti guiderà attraverso l'implementazione e l'applicazione delle principali tecniche di riduzione della dimensionalità discusse nelle lezioni teoriche. Esploreremo diversi dataset e vedremo come applicare e valutare vari metodi per ridurre la dimensionalità dei dati.

### Obiettivi dell'esercitazione:
- Implementare e applicare PCA, t-SNE, UMAP e Autoencoders
- Visualizzare i risultati della riduzione della dimensionalità
- Valutare la qualità delle rappresentazioni a bassa dimensione
- Confrontare le prestazioni delle diverse tecniche
- Applicare la riduzione della dimensionalità a casi di studio reali

## Configurazione dell'ambiente

Iniziamo importando le librerie necessarie per questa esercitazione.

In [ ]:
# Installare le librerie necessarie
!pip install umap-learn
!pip install tensorflow
!pip install scikit-learn matplotlib pandas seaborn

In [ ]:
# Importazione delle librerie necessarie
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
from sklearn.datasets import load_digits, fetch_openml, load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import time
import warnings
warnings.filterwarnings('ignore')

# Impostazioni di visualizzazione
plt.style.use('seaborn-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## Parte 1: Caricamento e Preparazione dei Dataset

Utilizzeremo diversi dataset per testare le tecniche di riduzione della dimensionalità.

In [ ]:
# Funzione per visualizzare i dataset
def plot_dataset(X, y, title="Dataset", discrete_cmap=True):
    plt.figure(figsize=(10, 8))
    
    if X.shape[1] > 2:
        # Se il dataset ha più di 2 dimensioni, utilizziamo PCA per visualizzarlo
        pca = PCA(n_components=2)
        X_2d = pca.fit_transform(X)
        plt.title(f"{title} (PCA 2D)\nVarianza spiegata: {pca.explained_variance_ratio_.sum():.2f}", fontsize=14)
    else:
        X_2d = X
        plt.title(title, fontsize=14)
    
    # Utilizziamo una colormap discreta o continua in base al tipo di target
    if discrete_cmap:
        scatter = plt.scatter(X_2d[:, 0], X_2d[:, 1], c=y, cmap='tab10', s=50, alpha=0.8)
        plt.colorbar(scatter, label='Classe')
    else:
        scatter = plt.scatter(X_2d[:, 0], X_2d[:, 1], c=y, cmap='viridis', s=50, alpha=0.8)
        plt.colorbar(scatter, label='Valore')
    
    plt.xlabel('Componente 1', fontsize=12)
    plt.ylabel('Componente 2', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# 1. Dataset Iris (bassa dimensionalità, 4 features)
iris = load_iris()
X_iris, y_iris = iris.data, iris.target
feature_names_iris = iris.feature_names
target_names_iris = iris.target_names

print(f"Dataset Iris: {X_iris.shape[0]} campioni, {X_iris.shape[1]} features")
print(f"Features: {feature_names_iris}")
print(f"Classi: {target_names_iris}")
plot_dataset(X_iris, y_iris, "Dataset Iris")

In [ ]:
# 2. Dataset Digits (media dimensionalità, 64 features)
digits = load_digits()
X_digits, y_digits = digits.data, digits.target

print(f"Dataset Digits: {X_digits.shape[0]} campioni, {X_digits.shape[1]} features")
print(f"Classi: {np.unique(y_digits)}")
plot_dataset(X_digits, y_digits, "Dataset Digits (Cifre 0-9)")

In [ ]:
# Visualizzare alcune immagini del dataset Digits
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
axes = axes.flatten()

for i in range(10):
    # Trovare un esempio di ogni cifra
    idx = np.where(y_digits == i)[0][0]
    axes[i].imshow(digits.images[idx], cmap='gray')
    axes[i].set_title(f"Cifra: {i}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# 3. Dataset MNIST (alta dimensionalità, 784 features)
# Carichiamo solo un sottoinsieme per velocizzare l'esercitazione
mnist = fetch_openml('mnist_784', version=1, parser='auto')
X_mnist = mnist.data.astype('float32').values[:5000]  # Prendiamo solo 5000 campioni
y_mnist = mnist.target.astype('int').values[:5000]

print(f"Dataset MNIST (sottoinsieme): {X_mnist.shape[0]} campioni, {X_mnist.shape[1]} features")
print(f"Classi: {np.unique(y_mnist)}")
plot_dataset(X_mnist, y_mnist, "Dataset MNIST (sottoinsieme)")

In [ ]:
# Visualizzare alcune immagini del dataset MNIST
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
axes = axes.flatten()

for i in range(10):
    # Trovare un esempio di ogni cifra
    idx = np.where(y_mnist == i)[0][0]
    axes[i].imshow(X_mnist[idx].reshape(28, 28), cmap='gray')
    axes[i].set_title(f"Cifra: {i}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## Parte 2: Principal Component Analysis (PCA)

Implementiamo e applichiamo PCA ai nostri dataset.

### 2.1 Implementazione di PCA

In [ ]:
def apply_pca(X, n_components=2, standardize=True, title="PCA"):
    # Standardizzare i dati se richiesto
    if standardize:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
    else:
        X_scaled = X.copy()
    
    # Applicare PCA
    start_time = time.time()
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_scaled)
    end_time = time.time()
    
    print(f"Tempo di esecuzione PCA: {end_time - start_time:.3f} secondi")
    print(f"Varianza spiegata dalle prime {n_components} componenti: {pca.explained_variance_ratio_.sum():.4f}")
    
    return X_pca, pca

In [ ]:
# Applicare PCA al dataset Iris
X_iris_pca, pca_iris = apply_pca(X_iris, n_components=2, title="PCA su Iris")

# Visualizzare i risultati
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_iris_pca[:, 0], X_iris_pca[:, 1], c=y_iris, cmap='tab10', s=50, alpha=0.8)
plt.colorbar(scatter, label='Classe')
plt.title(f"PCA su Iris\nVarianza spiegata: {pca_iris.explained_variance_ratio_.sum():.4f}", fontsize=14)
plt.xlabel(f"PC1 ({pca_iris.explained_variance_ratio_[0]:.4f})", fontsize=12)
plt.ylabel(f"PC2 ({pca_iris.explained_variance_ratio_[1]:.4f})", fontsize=12)

# Aggiungere i nomi delle classi
for i, target_name in enumerate(target_names_iris):
    plt.annotate(target_name,
                 xy=(X_iris_pca[y_iris == i, 0].mean(), X_iris_pca[y_iris == i, 1].mean()),
                 xytext=(3, 3),
                 textcoords='offset points',
                 ha='right',
                 va='bottom',
                 fontsize=12,
                 bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5))

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 2.2 Analisi delle Componenti Principali

In [ ]:
# Visualizzare i loadings delle componenti principali per Iris
plt.figure(figsize=(12, 6))
components = pd.DataFrame(pca_iris.components_, columns=feature_names_iris)
sns.heatmap(components, cmap='coolwarm', annot=True, fmt='.3f')
plt.title('Loadings delle Componenti Principali (Iris)', fontsize=14)
plt.ylabel('Componente Principale')
plt.tight_layout()
plt.show()

In [ ]:
# Visualizzare la varianza spiegata cumulativa
def plot_explained_variance(X, max_components=None, title="Varianza Spiegata Cumulativa"):
    if max_components is None:
        max_components = min(X.shape[0], X.shape[1])
    
    # Standardizzare i dati
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Applicare PCA con tutte le componenti
    pca = PCA(n_components=max_components)
    pca.fit(X_scaled)
    
    # Calcolare la varianza spiegata cumulativa
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    
    # Visualizzare i risultati
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.bar(range(1, max_components + 1), pca.explained_variance_ratio_, alpha=0.7)
    plt.step(range(1, max_components + 1), cumulative_variance, where='mid', color='red', alpha=0.7)
    plt.axhline(y=0.95, color='k', linestyle='--', alpha=0.7)
    plt.axhline(y=0.9, color='k', linestyle=':', alpha=0.7)
    plt.text(max_components/2, 0.96, '95% varianza spiegata', ha='center', va='bottom')
    plt.text(max_components/2, 0.91, '90% varianza spiegata', ha='center', va='bottom')
    plt.title('Varianza Spiegata per Componente')
    plt.xlabel('Componente Principale')
    plt.ylabel('Varianza Spiegata')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.plot(range(1, max_components + 1), cumulative_variance, 'o-', markersize=5)
    plt.axhline(y=0.95, color='k', linestyle='--', alpha=0.7)
    plt.axhline(y=0.9, color='k', linestyle=':', alpha=0.7)
    plt.text(max_components/2, 0.96, '95% varianza spiegata', ha='center', va='bottom')
    plt.text(max_components/2, 0.91, '90% varianza spiegata', ha='center', va='bottom')
    plt.title('Varianza Spiegata Cumulativa')
    plt.xlabel('Numero di Componenti')
    plt.ylabel('Varianza Spiegata Cumulativa')
    plt.grid(True, alpha=0.3)
    
    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # Trovare il numero di componenti necessarie per spiegare il 90% e il 95% della varianza
    n_components_90 = np.argmax(cumulative_variance >= 0.9) + 1
    n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
    
    print(f"Numero di componenti necessarie per spiegare il 90% della varianza: {n_components_90}")
    print(f"Numero di componenti necessarie per spiegare il 95% della varianza: {n_components_95}")
    
    return n_components_90, n_components_95

In [ ]:
# Analizzare la varianza spiegata per il dataset Iris
n_components_90_iris, n_components_95_iris = plot_explained_variance(X_iris, title="Varianza Spiegata (Iris)")

In [ ]:
# Analizzare la varianza spiegata per il dataset Digits
n_components_90_digits, n_components_95_digits = plot_explained_variance(X_digits, title="Varianza Spiegata (Digits)")

### 2.3 Visualizzazione delle Componenti Principali per Digits

In [ ]:
# Applicare PCA al dataset Digits
X_digits_pca, pca_digits = apply_pca(X_digits, n_components=2, title="PCA su Digits")

# Visualizzare i risultati
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_digits_pca[:, 0], X_digits_pca[:, 1], c=y_digits, cmap='tab10', s=50, alpha=0.8)
plt.colorbar(scatter, label='Cifra')
plt.title(f"PCA su Digits\nVarianza spiegata: {pca_digits.explained_variance_ratio_.sum():.4f}", fontsize=14)
plt.xlabel(f"PC1 ({pca_digits.explained_variance_ratio_[0]:.4f})", fontsize=12)
plt.ylabel(f"PC2 ({pca_digits.explained_variance_ratio_[1]:.4f})", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Visualizzare le prime componenti principali come immagini per Digits
n_components = 10
pca_digits_full = PCA(n_components=n_components).fit(X_digits)

fig, axes = plt.subplots(2, 5, figsize=(12, 5))
axes = axes.flatten()

for i in range(n_components):
    component = pca_digits_full.components_[i].reshape(8, 8)
    axes[i].imshow(component, cmap='viridis')
    axes[i].set_title(f"PC {i+1}")
    axes[i].axis('off')

plt.suptitle("Prime 10 Componenti Principali (Digits)", fontsize=14)
plt.tight_layout()
plt.subplots_adjust(top=0.85)
plt.show()

### 2.4 Ricostruzione delle immagini originali da PCA

In [ ]:
def reconstruct_from_pca(X, n_components_list, sample_indices=None, n_samples=5):
    # Standardizzare i dati
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Se non vengono forniti indici specifici, selezionare casualmente n_samples
    if sample_indices is None:
        sample_indices = np.random.choice(X.shape[0], n_samples, replace=False)
    else:
        n_samples = len(sample_indices)
    
    # Preparare la figura
    n_rows = n_samples
    n_cols = len(n_components_list) + 1  # +1 per l'immagine originale
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 2, n_rows * 2))
    
    # Per ogni campione
    for i, idx in enumerate(sample_indices):
        # Visualizzare l'immagine originale
        original_img = X[idx].reshape(8, 8)
        axes[i, 0].imshow(original_img, cmap='gray')
        axes[i, 0].set_title("Originale")
        axes[i, 0].axis('off')
        
        # Per ogni numero di componenti
        for j, n_comp in enumerate(n_components_list):
            # Applicare PCA e ricostruire
            pca = PCA(n_components=n_comp)
            X_pca = pca.fit_transform(X_scaled)
            X_reconstructed = pca.inverse_transform(X_pca)
            
            # Destandardizzare
            X_reconstructed = scaler.inverse_transform(X_reconstructed)
            
            # Visualizzare l'immagine ricostruita
            reconstructed_img = X_reconstructed[idx].reshape(8, 8)
            axes[i, j+1].imshow(reconstructed_img, cmap='gray')
            axes[i, j+1].set_title(f"{n_comp} comp.\n({pca.explained_variance_ratio_.sum():.2f})")
            axes[i, j+1].axis('off')
    
    plt.suptitle("Ricostruzione delle immagini con diverse componenti PCA", fontsize=14)
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.show()

In [ ]:
# Selezionare alcuni esempi di cifre diverse
sample_indices = [np.where(y_digits == i)[0][0] for i in range(5)]

# Ricostruire le immagini con diverse componenti PCA
reconstruct_from_pca(X_digits, [5, 10, 20, 30, 40], sample_indices=sample_indices)

## Parte 3: t-SNE (t-distributed Stochastic Neighbor Embedding)

Implementiamo e applichiamo t-SNE ai nostri dataset.

### 3.1 Implementazione di t-SNE

In [ ]:
def apply_tsne(X, n_components=2, perplexity=30, n_iter=1000, title="t-SNE"):
    # Standardizzare i dati
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Applicare t-SNE
    start_time = time.time()
    tsne = TSNE(n_components=n_components, perplexity=perplexity, n_iter=n_iter, random_state=42)
    X_tsne = tsne.fit_transform(X_scaled)
    end_time = time.time()
    
    print(f"Tempo di esecuzione t-SNE: {end_time - start_time:.3f} secondi")
    
    return X_tsne, tsne

In [ ]:
# Applicare t-SNE al dataset Iris
X_iris_tsne, tsne_iris = apply_tsne(X_iris, perplexity=10, title="t-SNE su Iris")

# Visualizzare i risultati
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_iris_tsne[:, 0], X_iris_tsne[:, 1], c=y_iris, cmap='tab10', s=50, alpha=0.8)
plt.colorbar(scatter, label='Classe')
plt.title("t-SNE su Iris", fontsize=14)
plt.xlabel("t-SNE 1", fontsize=12)
plt.ylabel("t-SNE 2", fontsize=12)

# Aggiungere i nomi delle classi
for i, target_name in enumerate(target_names_iris):
    plt.annotate(target_name,
                 xy=(X_iris_tsne[y_iris == i, 0].mean(), X_iris_tsne[y_iris == i, 1].mean()),
                 xytext=(3, 3),
                 textcoords='offset points',
                 ha='right',
                 va='bottom',
                 fontsize=12,
                 bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5))

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Applicare t-SNE al dataset Digits
X_digits_tsne, tsne_digits = apply_tsne(X_digits, title="t-SNE su Digits")

# Visualizzare i risultati
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_digits_tsne[:, 0], X_digits_tsne[:, 1], c=y_digits, cmap='tab10', s=50, alpha=0.8)
plt.colorbar(scatter, label='Cifra')
plt.title("t-SNE su Digits", fontsize=14)
plt.xlabel("t-SNE 1", fontsize=12)
plt.ylabel("t-SNE 2", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 3.2 Effetto dei parametri di t-SNE

In [ ]:
def explore_tsne_parameters(X, y, perplexity_values=[5, 30, 50], title="Effetto della Perplexity in t-SNE"):
    # Standardizzare i dati
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Preparare la figura
    fig, axes = plt.subplots(1, len(perplexity_values), figsize=(15, 5))
    
    # Per ogni valore di perplexity
    for i, perplexity in enumerate(perplexity_values):
        # Applicare t-SNE
        tsne = TSNE(n_components=2, perplexity=perplexity, random_state=42)
        X_tsne = tsne.fit_transform(X_scaled)
        
        # Visualizzare i risultati
        scatter = axes[i].scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, cmap='tab10', s=30, alpha=0.8)
        axes[i].set_title(f"Perplexity = {perplexity}")
        axes[i].set_xlabel("t-SNE 1")
        axes[i].set_ylabel("t-SNE 2")
        axes[i].grid(True, alpha=0.3)
    
    plt.colorbar(scatter, ax=axes, label='Classe')
    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)
    plt.show()

In [ ]:
# Esplorare l'effetto della perplexity su Iris
explore_tsne_parameters(X_iris, y_iris, perplexity_values=[5, 15, 30], title="Effetto della Perplexity in t-SNE (Iris)")

In [ ]:
# Esplorare l'effetto della perplexity su Digits
explore_tsne_parameters(X_digits, y_digits, perplexity_values=[5, 30, 50], title="Effetto della Perplexity in t-SNE (Digits)")

## Parte 4: UMAP (Uniform Manifold Approximation and Projection)

Implementiamo e applichiamo UMAP ai nostri dataset.

### 4.1 Implementazione di UMAP

In [ ]:
def apply_umap(X, n_components=2, n_neighbors=15, min_dist=0.1, title="UMAP"):
    # Standardizzare i dati
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Applicare UMAP
    start_time = time.time()
    reducer = umap.UMAP(n_components=n_components, n_neighbors=n_neighbors, min_dist=min_dist, random_state=42)
    X_umap = reducer.fit_transform(X_scaled)
    end_time = time.time()
    
    print(f"Tempo di esecuzione UMAP: {end_time - start_time:.3f} secondi")
    
    return X_umap, reducer

In [ ]:
# Applicare UMAP al dataset Iris
X_iris_umap, umap_iris = apply_umap(X_iris, n_neighbors=10, title="UMAP su Iris")

# Visualizzare i risultati
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_iris_umap[:, 0], X_iris_umap[:, 1], c=y_iris, cmap='tab10', s=50, alpha=0.8)
plt.colorbar(scatter, label='Classe')
plt.title("UMAP su Iris", fontsize=14)
plt.xlabel("UMAP 1", fontsize=12)
plt.ylabel("UMAP 2", fontsize=12)

# Aggiungere i nomi delle classi
for i, target_name in enumerate(target_names_iris):
    plt.annotate(target_name,
                 xy=(X_iris_umap[y_iris == i, 0].mean(), X_iris_umap[y_iris == i, 1].mean()),
                 xytext=(3, 3),
                 textcoords='offset points',
                 ha='right',
                 va='bottom',
                 fontsize=12,
                 bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5))

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Applicare UMAP al dataset Digits
X_digits_umap, umap_digits = apply_umap(X_digits, title="UMAP su Digits")

# Visualizzare i risultati
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_digits_umap[:, 0], X_digits_umap[:, 1], c=y_digits, cmap='tab10', s=50, alpha=0.8)
plt.colorbar(scatter, label='Cifra')
plt.title("UMAP su Digits", fontsize=14)
plt.xlabel("UMAP 1", fontsize=12)
plt.ylabel("UMAP 2", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 4.2 Effetto dei parametri di UMAP

In [ ]:
def explore_umap_parameters(X, y, n_neighbors_values=[5, 15, 30], min_dist_values=[0.01, 0.1, 0.5]):
    # Standardizzare i dati
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Preparare la figura
    fig, axes = plt.subplots(len(n_neighbors_values), len(min_dist_values), figsize=(15, 12))
    
    # Per ogni combinazione di parametri
    for i, n_neighbors in enumerate(n_neighbors_values):
        for j, min_dist in enumerate(min_dist_values):
            # Applicare UMAP
            reducer = umap.UMAP(n_components=2, n_neighbors=n_neighbors, min_dist=min_dist, random_state=42)
            X_umap = reducer.fit_transform(X_scaled)
            
            # Visualizzare i risultati
            scatter = axes[i, j].scatter(X_umap[:, 0], X_umap[:, 1], c=y, cmap='tab10', s=30, alpha=0.8)
            axes[i, j].set_title(f"n_neighbors={n_neighbors}, min_dist={min_dist}")
            axes[i, j].set_xlabel("UMAP 1")
            axes[i, j].set_ylabel("UMAP 2")
            axes[i, j].grid(True, alpha=0.3)
    
    plt.colorbar(scatter, ax=axes, label='Classe')
    plt.suptitle("Effetto dei parametri in UMAP", fontsize=14)
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    plt.show()

In [ ]:
# Esplorare l'effetto dei parametri di UMAP su Digits
explore_umap_parameters(X_digits, y_digits)

## Parte 5: Autoencoders per la Riduzione della Dimensionalità

Implementiamo e applichiamo Autoencoders ai nostri dataset.

### 5.1 Implementazione di un Autoencoder semplice

In [ ]:
def create_autoencoder(input_dim, encoding_dim=2):
    # Definire l'architettura dell'autoencoder
    input_layer = Input(shape=(input_dim,))
    
    # Encoder
    encoded = Dense(128, activation='relu')(input_layer)
    encoded = Dense(64, activation='relu')(encoded)
    encoded = Dense(encoding_dim, activation='linear')(encoded)
    
    # Decoder
    decoded = Dense(64, activation='relu')(encoded)
    decoded = Dense(128, activation='relu')(decoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)
    
    # Modelli
    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)
    
    # Compilare il modello
    autoencoder.compile(optimizer='adam', loss='mse')
    
    return autoencoder, encoder

In [ ]:
def apply_autoencoder(X, encoding_dim=2, epochs=50, batch_size=32, title="Autoencoder"):
    # Normalizzare i dati tra 0 e 1
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Creare l'autoencoder
    autoencoder, encoder = create_autoencoder(X.shape[1], encoding_dim)
    
    # Addestrare l'autoencoder
    start_time = time.time()
    history = autoencoder.fit(X_scaled, X_scaled, epochs=epochs, batch_size=batch_size, 
                             validation_split=0.2, verbose=0)
    end_time = time.time()
    
    print(f"Tempo di addestramento Autoencoder: {end_time - start_time:.3f} secondi")
    
    # Visualizzare la curva di loss
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Curva di Loss dell\'Autoencoder')
    plt.xlabel('Epoca')
    plt.ylabel('Loss (MSE)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Codificare i dati
    X_encoded = encoder.predict(X_scaled)
    
    return X_encoded, autoencoder, encoder, history

In [ ]:
# Applicare l'autoencoder al dataset Digits
X_digits_ae, autoencoder_digits, encoder_digits, history_digits = apply_autoencoder(X_digits, epochs=50, title="Autoencoder su Digits")

# Visualizzare i risultati
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_digits_ae[:, 0], X_digits_ae[:, 1], c=y_digits, cmap='tab10', s=50, alpha=0.8)
plt.colorbar(scatter, label='Cifra')
plt.title("Autoencoder su Digits", fontsize=14)
plt.xlabel("Dimensione Latente 1", fontsize=12)
plt.ylabel("Dimensione Latente 2", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 5.2 Ricostruzione delle immagini con Autoencoder

In [ ]:
def reconstruct_from_autoencoder(X, autoencoder, sample_indices=None, n_samples=5):
    # Normalizzare i dati tra 0 e 1
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Se non vengono forniti indici specifici, selezionare casualmente n_samples
    if sample_indices is None:
        sample_indices = np.random.choice(X.shape[0], n_samples, replace=False)
    else:
        n_samples = len(sample_indices)
    
    # Ricostruire le immagini
    X_reconstructed = autoencoder.predict(X_scaled)
    
    # Visualizzare le immagini originali e ricostruite
    fig, axes = plt.subplots(n_samples, 2, figsize=(6, n_samples * 3))
    
    for i, idx in enumerate(sample_indices):
        # Immagine originale
        axes[i, 0].imshow(X[idx].reshape(8, 8), cmap='gray')
        axes[i, 0].set_title("Originale")
        axes[i, 0].axis('off')
        
        # Immagine ricostruita
        axes[i, 1].imshow(X_reconstructed[idx].reshape(8, 8), cmap='gray')
        axes[i, 1].set_title("Ricostruita")
        axes[i, 1].axis('off')
    
    plt.suptitle("Ricostruzione delle immagini con Autoencoder", fontsize=14)
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.show()

In [ ]:
# Selezionare alcuni esempi di cifre diverse
sample_indices = [np.where(y_digits == i)[0][0] for i in range(5)]

# Ricostruire le immagini con l'autoencoder
reconstruct_from_autoencoder(X_digits, autoencoder_digits, sample_indices=sample_indices)

## Parte 6: Confronto tra Tecniche di Riduzione della Dimensionalità

Confrontiamo le prestazioni delle diverse tecniche di riduzione della dimensionalità.

In [ ]:
def compare_dimensionality_reduction(X, y, title="Confronto tra Tecniche di Riduzione della Dimensionalità"):
    # Standardizzare i dati
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Applicare PCA
    start_time = time.time()
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    pca_time = time.time() - start_time
    
    # Applicare t-SNE
    start_time = time.time()
    tsne = TSNE(n_components=2, random_state=42)
    X_tsne = tsne.fit_transform(X_scaled)
    tsne_time = time.time() - start_time
    
    # Applicare UMAP
    start_time = time.time()
    reducer = umap.UMAP(n_components=2, random_state=42)
    X_umap = reducer.fit_transform(X_scaled)
    umap_time = time.time() - start_time
    
    # Visualizzare i risultati
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # PCA
    scatter = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='tab10', s=30, alpha=0.8)
    axes[0].set_title(f"PCA\nTempo: {pca_time:.2f}s\nVarianza spiegata: {pca.explained_variance_ratio_.sum():.2f}")
    axes[0].set_xlabel("Componente 1")
    axes[0].set_ylabel("Componente 2")
    axes[0].grid(True, alpha=0.3)
    
    # t-SNE
    axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, cmap='tab10', s=30, alpha=0.8)
    axes[1].set_title(f"t-SNE\nTempo: {tsne_time:.2f}s")
    axes[1].set_xlabel("t-SNE 1")
    axes[1].set_ylabel("t-SNE 2")
    axes[1].grid(True, alpha=0.3)
    
    # UMAP
    axes[2].scatter(X_umap[:, 0], X_umap[:, 1], c=y, cmap='tab10', s=30, alpha=0.8)
    axes[2].set_title(f"UMAP\nTempo: {umap_time:.2f}s")
    axes[2].set_xlabel("UMAP 1")
    axes[2].set_ylabel("UMAP 2")
    axes[2].grid(True, alpha=0.3)
    
    plt.colorbar(scatter, ax=axes, label='Classe')
    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)
    plt.show()
    
    # Stampare i tempi di esecuzione
    print(f"Tempi di esecuzione:")
    print(f"PCA: {pca_time:.3f} secondi")
    print(f"t-SNE: {tsne_time:.3f} secondi")
    print(f"UMAP: {umap_time:.3f} secondi")
    
    return X_pca, X_tsne, X_umap

In [ ]:
# Confrontare le tecniche sul dataset Iris
X_pca_iris, X_tsne_iris, X_umap_iris = compare_dimensionality_reduction(X_iris, y_iris, title="Confronto su Iris")

In [ ]:
# Confrontare le tecniche sul dataset Digits
X_pca_digits, X_tsne_digits, X_umap_digits = compare_dimensionality_reduction(X_digits, y_digits, title="Confronto su Digits")

## Parte 7: Valutazione della Qualità della Riduzione della Dimensionalità

Valutiamo la qualità delle rappresentazioni a bassa dimensione utilizzando un classificatore.

In [ ]:
def evaluate_with_classifier(X_original, X_reduced_list, y, method_names, test_size=0.3):
    results = {}
    
    # Dividere i dati in training e test
    X_train_orig, X_test_orig, y_train, y_test = train_test_split(X_original, y, test_size=test_size, random_state=42)
    
    # Standardizzare i dati originali
    scaler = StandardScaler()
    X_train_orig = scaler.fit_transform(X_train_orig)
    X_test_orig = scaler.transform(X_test_orig)
    
    # Addestrare e valutare un classificatore sui dati originali
    knn_orig = KNeighborsClassifier(n_neighbors=5)
    knn_orig.fit(X_train_orig, y_train)
    y_pred_orig = knn_orig.predict(X_test_orig)
    accuracy_orig = accuracy_score(y_test, y_pred_orig)
    results['Originale'] = accuracy_orig
    
    # Per ogni metodo di riduzione della dimensionalità
    for X_reduced, method_name in zip(X_reduced_list, method_names):
        # Dividere i dati ridotti in training e test
        X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(X_reduced, y, test_size=test_size, random_state=42)
        
        # Addestrare e valutare un classificatore sui dati ridotti
        knn_red = KNeighborsClassifier(n_neighbors=5)
        knn_red.fit(X_train_red, y_train_red)
        y_pred_red = knn_red.predict(X_test_red)
        accuracy_red = accuracy_score(y_test_red, y_pred_red)
        results[method_name] = accuracy_red
    
    # Visualizzare i risultati
    plt.figure(figsize=(10, 6))
    plt.bar(results.keys(), results.values(), color='skyblue')
    plt.title('Accuratezza del Classificatore KNN', fontsize=14)
    plt.xlabel('Metodo di Riduzione della Dimensionalità')
    plt.ylabel('Accuratezza')
    plt.ylim(0, 1)
    
    # Aggiungere i valori sopra le barre
    for i, (key, value) in enumerate(results.items()):
        plt.text(i, value + 0.02, f"{value:.3f}", ha='center', va='bottom', fontsize=12)
    
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()
    
    return results

In [ ]:
# Valutare la qualità della riduzione della dimensionalità sul dataset Digits
results_digits = evaluate_with_classifier(X_digits, [X_pca_digits, X_tsne_digits, X_umap_digits, X_digits_ae], 
                                         y_digits, ['PCA', 't-SNE', 'UMAP', 'Autoencoder'])

## Parte 8: Caso di Studio - Visualizzazione di MNIST

Applichiamo le tecniche di riduzione della dimensionalità al dataset MNIST per visualizzare le cifre scritte a mano.

In [ ]:
# Applicare PCA a MNIST
X_mnist_pca, pca_mnist = apply_pca(X_mnist, title="PCA su MNIST")

# Visualizzare i risultati
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_mnist_pca[:, 0], X_mnist_pca[:, 1], c=y_mnist, cmap='tab10', s=5, alpha=0.8)
plt.colorbar(scatter, label='Cifra')
plt.title(f"PCA su MNIST\nVarianza spiegata: {pca_mnist.explained_variance_ratio_.sum():.4f}", fontsize=14)
plt.xlabel(f"PC1 ({pca_mnist.explained_variance_ratio_[0]:.4f})", fontsize=12)
plt.ylabel(f"PC2 ({pca_mnist.explained_variance_ratio_[1]:.4f})", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Applicare t-SNE a MNIST
X_mnist_tsne, tsne_mnist = apply_tsne(X_mnist, title="t-SNE su MNIST")

# Visualizzare i risultati
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_mnist_tsne[:, 0], X_mnist_tsne[:, 1], c=y_mnist, cmap='tab10', s=5, alpha=0.8)
plt.colorbar(scatter, label='Cifra')
plt.title("t-SNE su MNIST", fontsize=14)
plt.xlabel("t-SNE 1", fontsize=12)
plt.ylabel("t-SNE 2", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Applicare UMAP a MNIST
X_mnist_umap, umap_mnist = apply_umap(X_mnist, title="UMAP su MNIST")

# Visualizzare i risultati
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_mnist_umap[:, 0], X_mnist_umap[:, 1], c=y_mnist, cmap='tab10', s=5, alpha=0.8)
plt.colorbar(scatter, label='Cifra')
plt.title("UMAP su MNIST", fontsize=14)
plt.xlabel("UMAP 1", fontsize=12)
plt.ylabel("UMAP 2", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Parte 9: Applicazione Pratica - Compressione delle Immagini con PCA

Utilizziamo PCA per comprimere le immagini e valutare il compromesso tra dimensionalità e qualità.

In [ ]:
def compress_images_with_pca(X, n_components_list, sample_indices=None, n_samples=5):
    # Standardizzare i dati
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Se non vengono forniti indici specifici, selezionare casualmente n_samples
    if sample_indices is None:
        sample_indices = np.random.choice(X.shape[0], n_samples, replace=False)
    else:
        n_samples = len(sample_indices)
    
    # Calcolare il rapporto di compressione e l'errore di ricostruzione per ogni numero di componenti
    compression_ratios = []
    reconstruction_errors = []
    
    for n_comp in n_components_list:
        # Calcolare il rapporto di compressione
        original_size = X.shape[1]  # Dimensione originale
        compressed_size = n_comp * (1 + X.shape[1])  # Dimensione compressa (componenti + loadings)
        compression_ratio = original_size / compressed_size
        compression_ratios.append(compression_ratio)
        
        # Applicare PCA e ricostruire
        pca = PCA(n_components=n_comp)
        X_pca = pca.fit_transform(X_scaled)
        X_reconstructed = pca.inverse_transform(X_pca)
        
        # Destandardizzare
        X_reconstructed = scaler.inverse_transform(X_reconstructed)
        
        # Calcolare l'errore di ricostruzione (MSE)
        mse = np.mean((X - X_reconstructed) ** 2)
        reconstruction_errors.append(mse)
    
    # Visualizzare il compromesso tra compressione e qualità
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(n_components_list, compression_ratios, 'o-')
    plt.title('Rapporto di Compressione')
    plt.xlabel('Numero di Componenti')
    plt.ylabel('Rapporto di Compressione')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.plot(n_components_list, reconstruction_errors, 'o-')
    plt.title('Errore di Ricostruzione (MSE)')
    plt.xlabel('Numero di Componenti')
    plt.ylabel('MSE')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Visualizzare esempi di immagini compresse
    fig, axes = plt.subplots(n_samples, len(n_components_list) + 1, figsize=(2 * (len(n_components_list) + 1), 2 * n_samples))
    
    for i, idx in enumerate(sample_indices):
        # Visualizzare l'immagine originale
        axes[i, 0].imshow(X[idx].reshape(8, 8), cmap='gray')
        axes[i, 0].set_title("Originale")
        axes[i, 0].axis('off')
        
        # Visualizzare le immagini compresse
        for j, n_comp in enumerate(n_components_list):
            # Applicare PCA e ricostruire
            pca = PCA(n_components=n_comp)
            X_pca = pca.fit_transform(X_scaled)
            X_reconstructed = pca.inverse_transform(X_pca)
            
            # Destandardizzare
            X_reconstructed = scaler.inverse_transform(X_reconstructed)
            
            # Visualizzare l'immagine ricostruita
            axes[i, j+1].imshow(X_reconstructed[idx].reshape(8, 8), cmap='gray')
            axes[i, j+1].set_title(f"{n_comp} comp.\nCR: {compression_ratios[j]:.2f}")
            axes[i, j+1].axis('off')
    
    plt.suptitle("Compressione delle Immagini con PCA", fontsize=14)
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.show()
    
    return compression_ratios, reconstruction_errors

In [ ]:
# Selezionare alcuni esempi di cifre diverse
sample_indices = [np.where(y_digits == i)[0][0] for i in range(5)]

# Comprimere le immagini con PCA
compression_ratios, reconstruction_errors = compress_images_with_pca(X_digits, [5, 10, 20, 30, 40], sample_indices=sample_indices)

## Conclusioni

In questa esercitazione pratica, abbiamo esplorato diverse tecniche di riduzione della dimensionalità e le abbiamo applicate a vari dataset. Abbiamo visto come:

1. **PCA** è efficace per la compressione dei dati e la rimozione di caratteristiche ridondanti, ma è limitata a trasformazioni lineari.
2. **t-SNE** eccelle nella visualizzazione dei dati e nella preservazione della struttura locale, ma è computazionalmente costosa e non generalizzabile a nuovi dati.
3. **UMAP** offre un buon compromesso tra preservazione della struttura locale e globale, con tempi di esecuzione migliori rispetto a t-SNE.
4. **Autoencoders** sono flessibili e possono catturare relazioni non lineari complesse, ma richiedono più dati e tempo per l'addestramento.

Abbiamo anche imparato l'importanza di:
- Scegliere la tecnica appropriata in base all'obiettivo (visualizzazione, compressione, estrazione di caratteristiche)
- Valutare la qualità della riduzione della dimensionalità con metriche appropriate
- Considerare il compromesso tra dimensionalità e informazione preservata
- Ottimizzare i parametri delle diverse tecniche per ottenere i migliori risultati

La riduzione della dimensionalità è uno strumento potente nel machine learning, che ci permette di visualizzare, comprendere e lavorare con dati complessi in modo più efficace.

## Esercizi Aggiuntivi

1. Applica PCA, t-SNE e UMAP all'intero dataset MNIST e confronta i risultati.
2. Implementa un Autoencoder Variazionale (VAE) e confrontalo con un Autoencoder tradizionale.
3. Esplora l'effetto della normalizzazione dei dati sulle diverse tecniche di riduzione della dimensionalità.
4. Applica la riduzione della dimensionalità a un dataset di testi (ad esempio, utilizzando TF-IDF o word embeddings).
5. Implementa una versione semplificata di PCA da zero (senza utilizzare scikit-learn) e confronta i risultati con l'implementazione di scikit-learn.